In [1]:
#importing all dependencies
import numpy as np
import tweepy as tw
import tweepy.streaming
import requests
import base64

import pandas as pd

#Define your keys from the developer portal
consumer_key = '' # put your own key
consumer_secret_key = '' # put your own key

# authenticate
auth = tw.OAuthHandler(consumer_key, consumer_secret_key)
api = tw.API(auth, wait_on_rate_limit=True)


In [2]:
#Reformat the keys and encode them
key_secret = '{}:{}'.format(consumer_key, consumer_secret_key).encode('ascii')
#Transform from bytes to bytes that can be printed
b64_encoded_key = base64.b64encode(key_secret)
#Transform from bytes back into Unicode
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)
auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}
auth_data = {
    'grant_type': 'client_credentials'
}
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)
print(auth_resp.status_code)
access_token = auth_resp.json()['access_token']

200


In [4]:
trend_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

trend_params = {
    'id': 1,
}

trend_url = 'https://api.twitter.com/1.1/trends/place.json'  
trend_resp = requests.get(trend_url, headers=trend_headers, params=trend_params)

In [5]:
import json
from datetime import datetime
import ast

tweet_data = trend_resp.json()
print(json.dumps(tweet_data[0]['trends'], indent=3))

now = datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
date = now.strftime('%d')
hour = now.strftime('%H')

file_name = './trends_topic/' + year + '_' + month + '_' + date + '_' + hour + '.txt'
with open(file_name, 'w') as f:
    f.writelines(json.dumps(tweet_data[0]['trends'], indent=3))


[
   {
      "name": "#PFIBan",
      "url": "http://twitter.com/search?q=%23PFIBan",
      "promoted_content": null,
      "query": "%23PFIBan",
      "tweet_volume": 27857
   },
   {
      "name": "Kimbrel",
      "url": "http://twitter.com/search?q=Kimbrel",
      "promoted_content": null,
      "query": "Kimbrel",
      "tweet_volume": null
   },
   {
      "name": "\uc120\ubc30\uc758 \ud3c9\uac00",
      "url": "http://twitter.com/search?q=%22%EC%84%A0%EB%B0%B0%EC%9D%98+%ED%8F%89%EA%B0%80%22",
      "promoted_content": null,
      "query": "%22%EC%84%A0%EB%B0%B0%EC%9D%98+%ED%8F%89%EA%B0%80%22",
      "tweet_volume": null
   },
   {
      "name": "\ud3c9\uade0 \ud559\uc810",
      "url": "http://twitter.com/search?q=%22%ED%8F%89%EA%B7%A0+%ED%95%99%EC%A0%90%22",
      "promoted_content": null,
      "query": "%22%ED%8F%89%EA%B7%A0+%ED%95%99%EC%A0%90%22",
      "tweet_volume": null
   },
   {
      "name": "\ud6c4\ubc30\uc758 \ud3c9\uac00",
      "url": "http://twitter.com/search?q=%

In [6]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
detect('हिंदी_दिवस')

'mr'

In [7]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
topic_name = []
for i in range(50):
    topic = tweet_data[0]['trends'][i]['name']
    # print(detect(topic))
    topic_name.append(topic)
print(topic_name)

['#PFIBan', 'Kimbrel', '선배의 평가', '평균 학점', '후배의 평가', '#BhagatSingh', '爆破予告', '#サイスタアイドルとガシャ運試し', 'Messi', '#LMD5', 'Midnight Rain', '#DebateNaGlobo', 'Lazada', 'シーグリ', 'セノくん', 'Candace', 'Falcao', 'キャンディス', 'Cuadrado', 'アメスパ3', '第3回選抜ガチャ投票', 'Carrascal', 'Navratri Quiz', '에버랜드', 'Tata', '金木犀の香り', 'Colombia', 'EW04', 'Barrios', 'Sinisterra', '台風18号', '裏星座診断', 'Alzate', 'Alfaro', 'Cyno', 'たまアリ', 'Cláudio Castro', 'ウェンティ', '氷鷹北斗', 'Rosi', 'Lionel', '厚生年金', 'FGOメンテ', 'Hader', '初心LOVE', 'SDPI', 'Padres', 'Alexis Vega', '安倍元首相の顔面', 'Débora']


In [8]:
# search_query = "3racha -filter:retweets"
query_name = "Cyno"
search_query = query_name + ' -filter:retweets'
# search_query = query_name
print(search_query)

Cyno -filter:retweets


In [11]:
n = 1000
# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,tweet_mode='extended',
              lang="en").items(n)
# store the API responses in a list
tweets_copy = []

for tweet in tweets:
    tweets_copy.append(tweet)

print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 1000


In [ ]:
file_name = './tweets_copy/' + query_name + '_' + str(len(tweets_copy)) + '.txt'
with open(file_name, 'w', encoding="utf-8") as f:
    for status in tweets_copy:
        f.write(str(status._json) + '\n')

In [ ]:
import re
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        # text = api.get_status(id=tweet.id, tweet_mode='extended')
    except:
        pass
    
    try:
        text = tweet.retweeted_status.full_text[tweet.retweeted_status.full_text.find(': ')+1:]
    except:
        text = tweet.full_text

    # text = re.sub(r"http\S+", '', text)

    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                            'user_location': tweet.user.location,
                                            'user_description': tweet.user.description,
                                            'user_verified': tweet.user.verified,
                                            'date': tweet.created_at,
                                            'text': text, 
                                            'hashtags': [hashtags if hashtags else None],
                                            'source': tweet.source}))

tweets_df = tweets_df.reset_index(drop=True)
file_name_path = './tweet_trends/' + query_name + '_' + str(len(tweets_copy)) + '.csv'
tweets_df.to_csv(file_name_path)

In [ ]:
tweets_copy[10]._json
# print (tweets_copy[65].text[tweets_copy[65].text.find(': ')+1:])
# print(json.dumps(tweets_copy[0], indent=3))
# 1569970349898604544
# 1569970348870991878

{'created_at': 'Tue Sep 27 17:09:39 +0000 2022',
 'id': 1574808498126221312,
 'id_str': '1574808498126221312',
 'full_text': 'Turns out the angel and the devil on my shoulder are actually both just Elizabeth Olsen https://t.co/xoLk1GdVHc',
 'truncated': False,
 'display_text_range': [0, 87],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1574808493680259074,
    'id_str': '1574808493680259074',
    'indices': [88, 111],
    'media_url': 'http://pbs.twimg.com/media/FdrX-zcUcAIUGfV.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/FdrX-zcUcAIUGfV.jpg',
    'url': 'https://t.co/xoLk1GdVHc',
    'display_url': 'pic.twitter.com/xoLk1GdVHc',
    'expanded_url': 'https://twitter.com/itsashlyperez/status/1574808498126221312/photo/1',
    'type': 'photo',
    'sizes': {'large': {'w': 1000, 'h': 1250, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 544, 'h': 680, 'resize': 'fit'},
     'me

In [ ]:
tweets_df.head(5)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,lenakins 🪩,she/her,"it’s quite a strange feeling, isn’t it? never ...",False,2022-09-27 17:16:23+00:00,blank starring elizabeth olsen directed by amy...,None,Twitter for iPhone
1,Scott the Human,,"""He is a bear, then, with the shadow of a man.""",False,2022-09-27 17:16:10+00:00,"If Elizabeth Olsen wanted to stop being Wanda,...",None,Twitter Web App
2,BoxReview.com,,"All the Latest BREAKING News in the #Movie, #F...",False,2022-09-27 17:15:55+00:00,Elizabeth Olsen Reveals She Was Just as Shocke...,"[movies, movie, film, films, actor, actors, ac...",IFTTT
3,Variety,"Los Angeles, CA",The business of entertainment.,True,2022-09-27 17:14:45+00:00,"""I think the Avenger that will have done more ...",None,Twitter Media Studio
4,SPIDER WITCH FAN GIRL😎😍😼,"Assam, India",SPIDER MAN AND SCARLET WITCH \nFOR ME EVERYTHI...,False,2022-09-27 17:12:24+00:00,@jrvsscarlet Most powerful Avenger lol 🤣🤣🤣🤣🤣\n...,None,Twitter for Android


In [ ]:
tweets = tweets_df['text'].to_list()